In [1]:
import pandas as pd
import numpy as np

In [2]:
pop1 = pd.read_csv('/Users/caleboneel/Desktop/estimating-impact-of-opioids-2020-purlple-team/20_intermediate_files/countypopulations_clean.csv')

vitality = pd.read_csv('/Users/caleboneel/Desktop/estimating-impact-of-opioids-2020-purlple-team/10_code/od_deaths_state.csv')

In [3]:
mergepop = pop1.copy()
mergepop = mergepop.rename(columns = {'FIPS':'County Code', 'State Abbr':'State'}) 
mergepop.drop(['State Name','Unnamed: 0','_merge'], axis=1, inplace = True)
mergepop.drop(mergepop.loc[mergepop['Year']== 2000.0].index, inplace=True)
mergepop.drop(mergepop.loc[mergepop['Year']== 2001.0].index, inplace=True)
mergepop.drop(mergepop.loc[mergepop['Year']== 2002.0].index, inplace=True)
mergepop.drop(mergepop.loc[mergepop['Year']== 2016.0].index, inplace=True)
mergepop.drop(mergepop.loc[mergepop['Year']== 2017.0].index, inplace=True)
mergepop.drop(mergepop.loc[mergepop['Year']== 2018.0].index, inplace=True)
mergepop.drop(mergepop.loc[mergepop['Year']== 2019.0].index, inplace=True)
mergepop.drop(mergepop.loc[mergepop['State']== 'ND'].index, inplace=True)
mergepop

,County,Year,Population,County Code,State
3,Autauga County,2003.0,46800,1001,AL
4,Autauga County,2004.0,48366,1001,AL
5,Autauga County,2005.0,49676,1001,AL
6,Autauga County,2006.0,51328,1001,AL
7,Autauga County,2007.0,52405,1001,AL
...,...,...,...,...,...
62331,Weston County,2011.0,7142,56045,WY
62332,Weston County,2012.0,7077,56045,WY
62333,Weston County,2013.0,7136,56045,WY
62334,Weston County,2014.0,7138,56045,WY


In [4]:
vitality

,County,State,County Code,Year,Deaths
0,Acadia Parish,LA,22001,2003.0,11.0
1,Acadia Parish,LA,22001,2005.0,23.0
2,Acadia Parish,LA,22001,2006.0,19.0
3,Acadia Parish,LA,22001,2007.0,19.0
4,Acadia Parish,LA,22001,2009.0,11.0
...,...,...,...,...,...
7920,Yuma County,AZ,4027,2011.0,20.0
7921,Yuma County,AZ,4027,2012.0,32.0
7922,Yuma County,AZ,4027,2013.0,22.0
7923,Yuma County,AZ,4027,2014.0,35.0


In [5]:
combo = vitality.merge(mergepop, how = 'right', on = ['County Code', 'Year'], indicator = True)

In [6]:
death_coef = combo[combo['_merge'] == 'both']

In [10]:
death_coef = death_coef[['State_x', 'Year', 'Deaths', 'Population']]
death_coef
coef = death_coef.copy()

In [11]:
coef['death rate'] = coef['Deaths']/coef['Population']
coef.rename(columns = {'State_x':'State'}, inplace = True) 
coef

,State,Year,Deaths,Population,death rate
13,AL,2003.0,10.0,151509,0.000066
14,AL,2004.0,18.0,156266,0.000115
15,AL,2005.0,14.0,162183,0.000086
16,AL,2006.0,11.0,168121,0.000065
17,AL,2007.0,24.0,172404,0.000139
...,...,...,...,...,...
39698,WY,2012.0,12.0,78565,0.000153
39699,WY,2013.0,14.0,81103,0.000173
39700,WY,2014.0,17.0,81377,0.000209
39701,WY,2015.0,13.0,82137,0.000158


# Markdown incase reuse
wyoming = coef[coef['State_x'] == 'WY']
wyoming

wy = pop1.groupby(['State Abbr', 'Year'], as_index = False).sum()
wy1 = wy[wy['State Abbr'] == 'WY']
wy1

In [12]:
deathrate = coef.drop(['Deaths', 'Population'], axis=1)

In [13]:
combo['Merge?'] = combo['_merge']
combo.drop(['_merge'], axis=1, inplace = True)

In [14]:
combo.rename(columns = {'State_y':'State', 'County_y':'County'}, inplace = True)
combo.drop(['County_x', 'State_x'], axis = 1, inplace = True)

In [15]:
death = combo.merge(deathrate, how='left', on = ['State', 'Year'], indicator = True)

In [16]:
death['death rate'].isna().sum()
deathnan = death[death['_merge'] == 'left_only']

multiple = death.copy()
if ((multiple['Merge?'] == 'right_only') and (multiple['_merge'] == 'both')):
    deathcoef = round(multiple['Population'] * multiple['death rate'])
    multiple['Deaths'] = min(9, deathcoef)

multiple = death.copy()
multiple['deathcoef'] = round(multiple['Population'] * multiple['death rate'])
multiple.loc[multiple['Merge?'] == 'right_only', 'projdeaths'] = min(9, deathcoef)

In [17]:
deathnanyr = deathnan.groupby(['State','Year'], as_index = False).sum()
deathnanyr

,State,Year,County Code,Deaths,Population,death rate
0,MT,2004.0,1683136,0.0,930009,0.0
1,NE,2003.0,2891649,0.0,1738643,0.0
2,SD,2003.0,3040496,0.0,763729,0.0
3,SD,2004.0,3040496,0.0,770396,0.0
4,SD,2005.0,3040496,0.0,775493,0.0
5,SD,2006.0,3040496,0.0,783033,0.0
6,SD,2007.0,3040496,0.0,791623,0.0
7,SD,2008.0,3040496,0.0,799124,0.0
8,SD,2011.0,3040496,0.0,823579,0.0
9,SD,2012.0,3040496,0.0,833566,0.0


In [18]:
deathnanyr['State'].value_counts()


SD    8
WY    6
VT    6
NE    1
MT    1
Name: State, dtype: int64

In [19]:
multiple = death.copy()
missing =  multiple[multiple['State'].isin(['SD', 'VT', 'WY', 'NE', 'MT'])]  

In [20]:
SD = missing[missing['State']== 'SD']
VT = missing[missing['State']== 'VT']
WY = missing[missing['State']== 'WY']
NE = missing[missing['State']== 'NE']
MT = missing[missing['State']== 'MT']

In [21]:
SD = SD.groupby(['Year'], as_index = False).mean()
SDrate = SD['death rate'].mean()

VT = VT.groupby(['Year'], as_index = False).mean()
VTrate = VT['death rate'].mean()

WY = WY.groupby(['Year'], as_index = False).mean()
WYrate = WY['death rate'].mean()

NE = NE.groupby(['Year'], as_index = False).mean()
NErate = NE['death rate'].mean()

MT = MT.groupby(['Year'], as_index = False).mean()
MTrate = MT['death rate'].mean()

In [22]:
multcopy = multiple.copy()
multcopy

,County Code,Year,Deaths,County,Population,State,Merge?,death rate,_merge
0,1001,2003.0,NaN,Autauga County,46800,AL,right_only,0.000066,both
1,1001,2003.0,NaN,Autauga County,46800,AL,right_only,0.000105,both
2,1001,2003.0,NaN,Autauga County,46800,AL,right_only,0.000065,both
3,1001,2004.0,NaN,Autauga County,48366,AL,right_only,0.000115,both
4,1001,2004.0,NaN,Autauga County,48366,AL,right_only,0.000096,both
...,...,...,...,...,...,...,...,...,...
650192,56045,2014.0,NaN,Weston County,7138,WY,right_only,0.000209,both
650193,56045,2014.0,NaN,Weston County,7138,WY,right_only,0.000289,both
650194,56045,2015.0,NaN,Weston County,7208,WY,right_only,0.000249,both
650195,56045,2015.0,NaN,Weston County,7208,WY,right_only,0.000134,both


In [23]:
multcopy['death rate'] = np.where((multcopy.State == 'SD')&(multcopy['death rate'].isnull()),
                                 SDrate,
                                 multcopy['death rate'])

In [24]:
multcopy['death rate'] = np.where((multcopy.State == 'VT')&(multcopy['death rate'].isnull()),
                                 VTrate,
                                 multcopy['death rate'])

In [25]:
multcopy['death rate'] = np.where((multcopy.State == 'WY')&(multcopy['death rate'].isnull()),
                                 WYrate,
                                 multcopy['death rate'])

In [26]:
multcopy['death rate'] = np.where((multcopy.State == 'NE')&(multcopy['death rate'].isnull()),
                                 NErate,
                                 multcopy['death rate'])

In [27]:
multcopy['death rate'] = np.where((multcopy.State == 'MT')&(multcopy['death rate'].isnull()),
                                 MTrate,
                                 multcopy['death rate'])

In [28]:
SDnew = multcopy[multcopy['State'] == 'SD']

In [29]:
SDnew.groupby(['Year']).mean()

,County Code,Deaths,Population,death rate
Year,,,,
2003.0,46068.121212,NaN,11571.651515,0.000091
2004.0,46068.121212,NaN,11672.666667,0.000091
2005.0,46068.121212,NaN,11749.893939,0.000091
2006.0,46068.121212,NaN,11864.136364,0.000091
2007.0,46068.121212,NaN,11994.287879,0.000091
2008.0,46068.121212,NaN,12107.939394,0.000091
2009.0,46068.121212,14.0,12228.287879,0.000140
2010.0,46068.121212,11.0,12366.151515,0.000065
2011.0,46068.121212,NaN,12478.469697,0.000091


In [30]:
VTnew = multcopy[multcopy['State'] == 'VT']
VTnew

,County Code,Year,Deaths,County,Population,State,Merge?,death rate,_merge
593685,50001,2003.0,NaN,Addison County,36432,VT,right_only,0.000131,left_only
593686,50001,2004.0,NaN,Addison County,36580,VT,right_only,0.000131,left_only
593687,50001,2005.0,NaN,Addison County,36611,VT,right_only,0.000131,left_only
593688,50001,2006.0,NaN,Addison County,36758,VT,right_only,0.000098,both
593689,50001,2007.0,NaN,Addison County,36886,VT,right_only,0.000192,both
...,...,...,...,...,...,...,...,...,...
593932,50027,2014.0,NaN,Windsor County,55727,VT,right_only,0.000081,both
593933,50027,2015.0,12.0,Windsor County,55541,VT,both,0.000136,both
593934,50027,2015.0,12.0,Windsor County,55541,VT,both,0.000245,both
593935,50027,2015.0,12.0,Windsor County,55541,VT,both,0.000168,both


In [31]:
multcopy['death rate'].isnull().values.any()
multcopy

,County Code,Year,Deaths,County,Population,State,Merge?,death rate,_merge
0,1001,2003.0,NaN,Autauga County,46800,AL,right_only,0.000066,both
1,1001,2003.0,NaN,Autauga County,46800,AL,right_only,0.000105,both
2,1001,2003.0,NaN,Autauga County,46800,AL,right_only,0.000065,both
3,1001,2004.0,NaN,Autauga County,48366,AL,right_only,0.000115,both
4,1001,2004.0,NaN,Autauga County,48366,AL,right_only,0.000096,both
...,...,...,...,...,...,...,...,...,...
650192,56045,2014.0,NaN,Weston County,7138,WY,right_only,0.000209,both
650193,56045,2014.0,NaN,Weston County,7138,WY,right_only,0.000289,both
650194,56045,2015.0,NaN,Weston County,7208,WY,right_only,0.000249,both
650195,56045,2015.0,NaN,Weston County,7208,WY,right_only,0.000134,both


In [32]:
#multcopy.drop(['deathcoef'], axis=1, inplace = True)

In [33]:
multcopy['deathcoef'] = round(multcopy['Population'] * multcopy['death rate'])


In [34]:
multcopy['projdeaths'] = np.where((multcopy.deathcoef < 9),
                                 multcopy.deathcoef,
                                 9)

In [35]:
multcopy
WA = multcopy[multcopy['State'] == 'WA']
WA

,County Code,Year,Deaths,County,Population,State,Merge?,death rate,_merge,deathcoef,projdeaths
623417,53001,2003.0,NaN,Adams County,16612,WA,right_only,0.000065,both,1.0,1.0
623418,53001,2003.0,NaN,Adams County,16612,WA,right_only,0.000107,both,2.0,2.0
623419,53001,2003.0,NaN,Adams County,16612,WA,right_only,0.000116,both,2.0,2.0
623420,53001,2003.0,NaN,Adams County,16612,WA,right_only,0.000096,both,2.0,2.0
623421,53001,2003.0,NaN,Adams County,16612,WA,right_only,0.000119,both,2.0,2.0
...,...,...,...,...,...,...,...,...,...,...,...
631134,53077,2015.0,26.0,Yakima County,247800,WA,both,0.000165,both,41.0,9.0
631135,53077,2015.0,26.0,Yakima County,247800,WA,both,0.000188,both,47.0,9.0
631136,53077,2015.0,26.0,Yakima County,247800,WA,both,0.000097,both,24.0,9.0
631137,53077,2015.0,26.0,Yakima County,247800,WA,both,0.000109,both,27.0,9.0


In [36]:
multcopy['CalculatedDeaths'] = np.where((multcopy.Deaths.isnull()),
                                 multcopy.projdeaths,
                                 multcopy.Deaths)

In [37]:
multcopy
WA = multcopy[multcopy['State'] == 'WA']
wa = WA[WA['Deaths'].isnull()]
wa.CalculatedDeaths.sum()

18818.0

In [38]:
assert len(multcopy) == len(mergepop)
assert multcopy.projdeaths.isnull().values.any() == False

AssertionError: 

In [39]:
multcopy.drop(['Merge?', '_merge'], axis=1, inplace = True)

In [40]:
vitalitydf = multcopy.copy()

In [41]:
vitalitydf.to_csv('vitalitydf_allcounties.csv', index=False)